In [2]:
from utils import *
import tensorflow as tf
from sklearn.model_selection import train_test_split
import time

In [3]:
trainset = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

['negative', 'positive']
10662
10662


In [4]:
ONEHOT = np.zeros((len(trainset.data),len(trainset.target_names)))
ONEHOT[np.arange(len(trainset.data)),trainset.target] = 1.0
train_X, test_X, train_Y, test_Y, train_onehot, test_onehot = train_test_split(trainset.data, 
                                                                               trainset.target, 
                                                                               ONEHOT, test_size = 0.2)

In [5]:
concat = ' '.join(trainset.data).split()
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])

vocab from size: 20332
Most common words [('film', 1453), ('movie', 1270), ('one', 727), ('like', 721), ('story', 477), ('much', 386)]
Sample data [24, 143, 5347, 1785, 2417, 260, 4, 4373, 28, 10158] ['may', 'think', 'figured', 'con', 'players', 'debut', 'film', 'argentine', 'director', 'fabian']


In [6]:
GO = dictionary['GO']
PAD = dictionary['PAD']
EOS = dictionary['EOS']
UNK = dictionary['UNK']

In [7]:
class Model:
    def __init__(self, size_layer, num_layers, embedded_size,
                 dict_size, dimension_output, learning_rate,
                filter_sizes, pooling_size):
        
        def cells(reuse=False):
            return tf.nn.rnn_cell.LSTMCell(size_layer,initializer=tf.orthogonal_initializer(),reuse=reuse)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.float32, [None, dimension_output])
        encoder_embeddings = tf.Variable(tf.random_uniform([dict_size, embedded_size], -1, 1))
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        encoder_embedded = tf.expand_dims(encoder_embedded,-1)
        
        pooled_outputs = []
        reduce_size = int(np.ceil(maxlen / pooling_size))
        for i in filter_sizes:
            #w就是filter 维度[filter_height, filter_width, in_channels, out_channels]
            w = tf.Variable(tf.truncated_normal([i, embedded_size, 1, dimension_output], stddev=0.1))
            b = tf.Variable(tf.truncated_normal([dimension_output], stddev = 0.01))
            
            self.conv = tf.nn.relu(tf.nn.conv2d(encoder_embedded, w, strides=[1, 1, 1, 1],padding="VALID") + b)
            
            pooled = tf.nn.max_pool(self.conv,ksize=[1, pooling_size, 1, 1],
                                    strides=[1, pooling_size, 1, 1],padding='VALID')
            
            pooled = tf.reshape(pooled, [-1, reduce_size -1, dimension_output])
            pooled_outputs.append(pooled)
        h_pool = tf.concat(pooled_outputs, 2)
        
        rnn_cells = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)])
        outputs, _ = tf.nn.dynamic_rnn(rnn_cells, h_pool, dtype = tf.float32)
        W = tf.get_variable('w',shape=(size_layer, dimension_output),initializer=tf.orthogonal_initializer())
        b = tf.get_variable('b',shape=(dimension_output),initializer=tf.zeros_initializer())
        self.logits = tf.matmul(outputs[:, -1], W) + b
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = self.logits, labels = self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        correct_pred = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [8]:
size_layer = 128
num_layers = 2
embedded_size = 128
dimension_output = len(trainset.target_names)
learning_rate = 1e-3
maxlen = 50
batch_size = 128

In [9]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(size_layer,num_layers,embedded_size,vocabulary_size+4,dimension_output,learning_rate,
             [3, 3, 3], 5)
sess.run(tf.global_variables_initializer())

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [11]:
EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 5, 0, 0, 0
while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n'%(EPOCH))
        break
        
    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    for i in range(0, (len(train_X) // batch_size) * batch_size, batch_size):
        batch_x = str_idx(train_X[i:i+batch_size],dictionary,maxlen)
        
        #调试代码
        feed_dict = {model.X : batch_x, model.Y : train_onehot[i:i+batch_size]}
        
        acc, loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                           feed_dict = {model.X : batch_x, model.Y : train_onehot[i:i+batch_size]})
        train_loss += loss
        train_acc += acc
    
    for i in range(0, (len(test_X) // batch_size) * batch_size, batch_size):
        batch_x = str_idx(test_X[i:i+batch_size],dictionary,maxlen)
        acc, loss = sess.run([model.accuracy, model.cost], 
                           feed_dict = {model.X : batch_x, model.Y : test_onehot[i:i+batch_size]})
        test_loss += loss
        test_acc += acc
    
    train_loss /= (len(train_X) // batch_size)
    train_acc /= (len(train_X) // batch_size)
    test_loss /= (len(test_X) // batch_size)
    test_acc /= (len(test_X) // batch_size)
    
    if test_acc > CURRENT_ACC:
        print('epoch: %d, pass acc: %f, current acc: %f'%(EPOCH,CURRENT_ACC, test_acc))
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time()-lasttime)
    print('epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'%(EPOCH,train_loss,
                                                                                          train_acc,test_loss,
                                                                                          test_acc))
    EPOCH += 1

epoch: 0, pass acc: 0.000000, current acc: 0.530273
time taken: 1.3522133827209473
epoch: 0, training loss: 0.693355, training acc: 0.498816, valid loss: 0.691207, valid acc: 0.530273

epoch: 1, pass acc: 0.530273, current acc: 0.576660
time taken: 0.9518313407897949
epoch: 1, training loss: 0.655117, training acc: 0.605587, valid loss: 0.694892, valid acc: 0.576660

epoch: 2, pass acc: 0.576660, current acc: 0.602051
time taken: 0.9466142654418945
epoch: 2, training loss: 0.516276, training acc: 0.727628, valid loss: 0.799324, valid acc: 0.602051

epoch: 3, pass acc: 0.602051, current acc: 0.617188
time taken: 0.9406802654266357
epoch: 3, training loss: 0.364851, training acc: 0.822562, valid loss: 1.040798, valid acc: 0.617188

epoch: 4, pass acc: 0.617188, current acc: 0.619629
time taken: 0.9495558738708496
epoch: 4, training loss: 0.245289, training acc: 0.882339, valid loss: 1.515840, valid acc: 0.619629

time taken: 0.9463872909545898
epoch: 5, training loss: 0.179140, training 

In [12]:
logits = sess.run(model.logits, feed_dict={model.X:str_idx(test_X,dictionary,maxlen)})
print(metrics.classification_report(test_Y, np.argmax(logits,1), target_names = trainset.target_names))

              precision    recall  f1-score   support

    negative       0.61      0.65      0.63      1064
    positive       0.63      0.58      0.60      1069

    accuracy                           0.62      2133
   macro avg       0.62      0.62      0.62      2133
weighted avg       0.62      0.62      0.62      2133

